In [18]:
import pandas as pd
from sqlalchemy import create_engine
price_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_price_yf.db')
ticker_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
ticker_df = pd.read_sql('TOTAL', con=ticker_engine)
ticker_list = ticker_df.Symbol.to_list()
import tushare as ts
pro = ts.pro_api(token='28ce867873a056747209a2ef0ff53d98ed850903fd954417ac69553b')
#获取富时中国50指数
df = pro.index_global(ts_code='SPX', start_date='20000201', end_date='20220913')
# df.to_csv('SPX.csv',index=None)
# df = pd.read_csv('SPX.csv')
df[df.trade_date == '20181003']

,ts_code,trade_date,open,close,high,low,pre_close,change,pct_chg,swing,vol
978,SPX,20181003,2931.69,2925.51,2939.86,2921.36,2923.43,2.08,0.07,0.63,2101580.0


In [19]:
downtrend_list =[['2022-01-03', '2022-06-21'],
                ['2020-02-20', '2022-03-23'],
                ['2018-10-03', '2018-12-26']]
start_date = downtrend_list[2][0]
end_date = downtrend_list[2][1]
start_index = df[df.trade_date == start_date.replace('-','')].index.values[0]
end_index = df[df.trade_date == end_date.replace('-','')].index.values[0]
last_count = abs(start_index - end_index)
last_count

57

In [20]:
ticker = 'AAPL'
ticker_df = pd.read_sql('{}'.format(ticker),con=price_engine)
ticker_df.rename(columns={'hgih':'high'},inplace=True)
# 初始价格
start_price = ticker_df[ticker_df.datetime == start_date].close.values[0]
# 初始价格时的index
tstart_index = ticker_df[ticker_df.datetime == start_date].index.values[0]
# 下降中的最低价
lowest_price = min(ticker_df[tstart_index:tstart_index+int(last_count*1.33)].low)
# 最低价时的index
lowest_index = ticker_df[ticker_df.low == lowest_price].index.values[0]
# 下降率
decline_rate = round((start_price - lowest_price) / start_price, 3)
# 下降时间
tlast_count = abs(lowest_index - tstart_index)
# 更新df的范围，起始点为最低价
ticker_df = ticker_df[lowest_index:]

In [21]:
ticker_df

,open,high,low,close,Adj Close,volume,datetime
9596,35.994999,36.430000,35.500000,35.547501,34.414234,365248800,2019-01-03
9597,36.132500,37.137501,35.950001,37.064999,35.883358,234428400,2019-01-04
9598,37.174999,37.207500,36.474998,36.982498,35.803486,219111200,2019-01-07
9599,37.389999,37.955002,37.130001,37.687500,36.486015,164101200,2019-01-08
9600,37.822498,38.632500,37.407501,38.327499,37.105606,180396400,2019-01-09
...,...,...,...,...,...,...,...
10480,145.259995,147.550003,145.000000,147.039993,147.039993,64493200,2022-07-08
10481,145.669998,146.639999,143.779999,144.869995,144.869995,63141600,2022-07-11
10482,145.759995,148.449997,145.050003,145.860001,145.860001,77588800,2022-07-12
10483,142.990005,146.449997,142.119995,145.490005,145.490005,71185600,2022-07-13


In [22]:
# 获取到达原价75%时的index和价格
if ticker_df[ticker_df.close >= start_price * 0.75].empty != True:
    rev_h_index = ticker_df[ticker_df.close >= start_price * 0.75].index.values[0]
    rev_h_price = ticker_df[ticker_df.close >= start_price * 0.75].close.values[0]
else:
    rev_h_index = 9999
    rev_h_price = 9999
# 获取到达最低价125%时的index
if ticker_df[ticker_df.close >= lowest_price * 1.25].empty != True:
    rev_l_index = ticker_df[ticker_df.close >= lowest_price * 1.25].index.values[0]
    rev_l_price = ticker_df[ticker_df.close >= lowest_price * 1.25].close.values[0]
else: 
    rev_l_index = 9999
    rev_l_price = 9999
# 分别计算到达的时间
rev_h_count = abs(rev_h_index - lowest_index)
rev_l_count = abs(rev_l_index - lowest_index)
# 进场点（用于计算收益，取小的index）
if rev_h_index == rev_l_index == 9999:
    print('can not continue')
else:
    if rev_l_count < rev_h_count:
        enter_point = rev_l_count
        enter_price = rev_l_price
    else:
        enter_point = rev_h_count
        enter_price = rev_h_price

In [25]:
ticker_df

,open,high,low,close,Adj Close,volume,datetime
9596,35.994999,36.430000,35.500000,35.547501,34.414234,365248800,2019-01-03
9597,36.132500,37.137501,35.950001,37.064999,35.883358,234428400,2019-01-04
9598,37.174999,37.207500,36.474998,36.982498,35.803486,219111200,2019-01-07
9599,37.389999,37.955002,37.130001,37.687500,36.486015,164101200,2019-01-08
9600,37.822498,38.632500,37.407501,38.327499,37.105606,180396400,2019-01-09
...,...,...,...,...,...,...,...
10480,145.259995,147.550003,145.000000,147.039993,147.039993,64493200,2022-07-08
10481,145.669998,146.639999,143.779999,144.869995,144.869995,63141600,2022-07-11
10482,145.759995,148.449997,145.050003,145.860001,145.860001,77588800,2022-07-12
10483,142.990005,146.449997,142.119995,145.490005,145.490005,71185600,2022-07-13


In [23]:
enter_point

22

In [24]:
ticker_df.iloc[22]

open             43.215
high              43.77
low           43.087502
close         43.544998
Adj Close     42.156769
volume        144406400
datetime     2019-02-05
Name: 9618, dtype: object

In [26]:
(43.54 - start_price) / start_price

-0.24953680138129286

In [27]:
(43.54 - 35.5) /35.5

0.2264788732394366